In [1]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d sanikamal/rock-paper-scissors-dataset

100% 452M/452M [00:04<00:00, 144MB/s]
100% 452M/452M [00:04<00:00, 113MB/s]


In [3]:
!unzip rock-paper-scissors-dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: Rock-Paper-Scissors/train/paper/paper04-118.png  
  inflating: Rock-Paper-Scissors/train/paper/paper04-119.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-000.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-001.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-002.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-003.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-004.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-005.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-006.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-007.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-008.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-009.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-010.png  
  inflating: Rock-Paper-Scissors/train/paper/paper05-011.png  
  inflating: Rock-Paper-Scissors/train/paper/paper

In [31]:
import tensorflow as tf
import os
import re

train_dir = "/content/rock-paper-scissors/Rock-Paper-Scissors/train"
test_dir = "/content/rock-paper-scissors/Rock-Paper-Scissors/test"
validation_dir = "/content/rock-paper-scissors/Rock-Paper-Scissors/validation"


image_size = (224, 224)
batch_size = 32


train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size
)


test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    batch_size=batch_size
)


validation_images = []
validation_labels = []
for filename in os.listdir(validation_dir):
    category = re.sub(r'-\w+', '', filename.split('.')[0])
    category = re.sub(r'\d+', '', category)
    img = tf.keras.preprocessing.image.load_img(
        os.path.join(validation_dir, filename), target_size=image_size
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    validation_images.append(img_array)
    validation_labels.append(category)

validation_dataset = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

Found 2520 files belonging to 3 classes.
Found 372 files belonging to 3 classes.


In [32]:
import tensorflow as tf

categories = train_dataset.class_names
num_classes = len(categories)
category_to_index = {category: index for index, category in enumerate(categories)}


def process_labels(x, y):
    y = tf.strings.regex_replace(y, r'-\w+', '')
    y = tf.strings.regex_replace(y, r'\d+', '')
    y = tf.strings.regex_replace(y, r'-', '')
    index = tf.py_function(lambda y: category_to_index[y.numpy().decode("utf-8")], [y], tf.int64)
    one_hot = tf.one_hot(index, num_classes)
    return x, one_hot

validation_dataset = validation_dataset.map(process_labels)

print(validation_dataset)

<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>


tf.Tensor([1. 0. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([1. 0. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([1. 0. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([1. 0. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)
tf.Tensor([1. 0. 0.], shape=(3,), dtype=float32)
tf.Tensor([0. 1. 0.], shape=(3,), dtype=float32)
tf.Tensor([1. 0. 0.]

In [33]:
validation_dataset = validation_dataset.map(lambda x, y: (tf.expand_dims(x, axis=0), y))

In [36]:
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(16, 3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.5)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Dropout(0.5)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',
              metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="rps_2.h5",
        save_best_only=True,
        monitor="val_loss")
]

history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/10
79/79 [==============================] - 9s 90ms/step - loss: 1.8763 - accuracy: 0.4440 - val_loss: 0.9207 - val_accuracy: 0.4545
Epoch 2/10
79/79 [==============================] - 6s 71ms/step - loss: 0.6842 - accuracy: 0.7222 - val_loss: 0.4754 - val_accuracy: 0.7273
Epoch 3/10
79/79 [==============================] - 9s 103ms/step - loss: 0.3794 - accuracy: 0.8726 - val_loss: 0.3718 - val_accuracy: 0.8485
Epoch 4/10
79/79 [==============================] - 7s 80ms/step - loss: 0.2743 - accuracy: 0.9123 - val_loss: 0.4414 - val_accuracy: 0.7273
Epoch 5/10
79/79 [==============================] - 7s 72ms/step - loss: 0.2398 - accuracy: 0.9171 - val_loss: 0.1981 - val_accuracy: 0.9394
Epoch 6/10
79/79 [==============================] - 7s 87ms/step - loss: 0.3243 - accuracy: 0.9385 - val_loss: 0.5016 - val_accuracy: 0.7273
Epoch 7/10
79/79 [==============================] - 7s 88ms/step - loss: 0.1757 - accuracy: 0.9472 - val_loss: 0.1675 - val_accuracy: 0.9394
Epoch 8/10
7

In [37]:
test_model = keras.models.load_model('rps_2.h5')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

12/12 [==============================] - 1s 45ms/step - loss: 0.5102 - accuracy: 0.8522
Test accuracy: 0.852
